In [122]:
import numpy as np
import pandas as pd

allfiles = ['b_should_be_easy.in', 'c_no_hurry.in', 'd_metropolis.in', 'e_high_bonus.in']
alldfs = []

for file in allfiles:
    with open(file, 'r') as f:
        df = pd.DataFrame([ride.split(" ") for ride in f.readlines()],
                      dtype=float,
                      columns=['start_row', 'start_col', 'end_row', 'end_col', 'time_earliest', 'time_latest'])
        alldfs.append(df)

b_df, c_df, d_df, e_df = alldfs

"""
LET'S WORK WITH THE FIRST DATASET: b_should_be_easy.in
"""

# save useful variables
b_NUM_ROWS, b_NUM_COLS, b_NUM_VEHICLES, b_NUM_RIDES, b_NUM_BONUS, b_NUM_STEPS = b_df.loc[0, :]

# remove first row
b_df = b_df.loc[1:,:]

# compute distance
b_df['distance'] = np.abs(b_df.end_col - b_df.start_col) + np.abs(b_df.end_row - b_df.start_row)

# sort by earliest time and distance
b_df.sort_values(by=['time_earliest', 'distance'], inplace=True)

# add ride_id field
b_df = b_df.reset_index(drop=True)
b_df = b_df.reset_index().rename({'index': 'ride_id'}, axis=1)

b_df=b_df.astype(int)
print(b_df.head())


   ride_id  start_row  start_col  end_row  end_col  time_earliest  \
0        0        518        656      201      494            186   
1        1        666        188      504      995            208   
2        2        502        138       42      160            228   
3        3        395         43      296      955            244   
4        4        662        421      314      445            281   

   time_latest  distance  
0          712       479  
1         2340       969  
2         1244       482  
3         3620      1011  
4          656       372  


/home/alvaro/.local/lib/python3.5/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alvaro/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [123]:
b_df.head()

,ride_id,start_row,start_col,end_row,end_col,time_earliest,time_latest,distance
0,0,518,656,201,494,186,712,479
1,1,666,188,504,995,208,2340,969
2,2,502,138,42,160,228,1244,482
3,3,395,43,296,955,244,3620,1011
4,4,662,421,314,445,281,656,372


In [124]:
#df_vehicles lambda function to each ride yet to complete
def stepSubstract(row):
#     print('start sub',row)
    row['steps_available']=row['steps_available']-1
    row['steps_early']=row['steps_early']-1
    if row['steps_early']==0: #run ends
#         print(row)
        if isinstance(row['ride_id'],list):
            row['ride_id']=row['ride_id'].append(row['rideToComplete'])
        else:
            row['ride_id']=[row['ride_id']]
        row['rideToComplete']=-1
        if row['steps_available']==0:
            row['status']='stop'
        else: 
            row['status']='available'
#     print('end sub',row)
    return row
            
def assignRideToVehicle(dfVehicles,rideRow,dfRides):
    #get first vehicle capable
    vehicleAvailables=dfVehicles[(dfVehicles['status']=='available') &\
    ((dfVehicles['steps_available']==-1)|\
     (dfVehicles['steps_available']>= rideRow.distance))].vehicleID.values
    if len(vehicleAvailables)>0:
        for vID in vehicleAvailables:
    #assign it rideID
            if dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]==-1:
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'rideToComplete']=rideRow['ride_id']
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available']=rideRow['time_latest']
#                 print('in1',dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>0)
#                 print('in2',dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>rideRow['time_earliest'])
# mind it may still have steps_available to 0
            if (dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>0)&(dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>rideRow['time_earliest']):
#                 print('in3')            
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'rideToComplete']=dfVehicles.loc[dfVehicles['vehicleID']==vID,'rideToComplete'].apply(lambda x: x.append(rideRow['ride_id']) if isinstance(x,list) else [rideRow['ride_id']])
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_early']=rideRow['time_earliest']
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'status']='start'
            #         print(dfVehicles.head())
    #drop ride from ridesDF
#                 print('fanculo',dfVehicles.loc[dfVehicles['vehicleID']==vID])
    
#                 print('fanculo2',rideRow.head())
                dfRides=dfRides.drop(rideRow['ride_id'])
#                 print('fanculo2',rideRow.head())
            
                return

In [125]:
df_vehicles=pd.DataFrame({'vehicleID':[],
                          'ride_id':[],
                          'steps_available':[],
                          'steps_early':[],
                          'status':[],
                          'rideToComplete':[]})
# rowVehicle='vehicleID':''.'rideID':[].'steps_available':'','status':'','rideToComplete':''

df_vehicles['vehicleID'] = range(int(b_NUM_VEHICLES))
df_vehicles['status'] = 'available'
df_vehicles['steps_available'] = -1
df_vehicles['steps_early'] = -1
df_vehicles['rideToComplete'] = -1

df_vehicles['ride_id'] = -1

In [126]:
b_df.head()

,ride_id,start_row,start_col,end_row,end_col,time_earliest,time_latest,distance
0,0,518,656,201,494,186,712,479
1,1,666,188,504,995,208,2340,969
2,2,502,138,42,160,228,1244,482
3,3,395,43,296,955,244,3620,1011
4,4,662,421,314,445,281,656,372


In [ ]:
for step in range(int(b_NUM_STEPS)):
    rides_to_assign = b_df[b_df.time_earliest == step]
    if len(rides_to_assign)>0:
#         print(rides_to_assign.head())
        for index, rowRide in rides_to_assign.iterrows():
            assignRideToVehicle(df_vehicles, rowRide, rides_to_assign)
    if(len(df_vehicles[df_vehicles['status']=='start'])>0):
        for index, rowV in df_vehicles[df_vehicles['status']=='start'].iterrows():
#             df_vehicles.ix[index,:]=stepSubstract(rowV)
            df_vehicles.loc[index,'steps_available']=rowV['steps_available']-1
            df_vehicles.loc[index,'steps_early']=rowV['steps_early']-1
            if df_vehicles.loc[index,'steps_early']==0: #run ends
        #         print(row)
                if isinstance(rowV['ride_id'],list):
                    df_vehicles.loc[index,'ride_id']=rowV['ride_id'].append(rowV['rideToComplete'])
                else:
                    df_vehicles.loc[index,'ride_id']=rowV['ride_id']
                df_vehicles.loc[index,'rideToComplete']=-1
                if rowV['steps_available']==0:
                    df_vehicles.loc[index,'status']='stop'
                else: 
                    df_vehicles.loc[index,'status']='available'
#         break
#     break

In [ ]:
df_vehicles

In [ ]:
b_df.shape